In [1]:
import numpy as np 
import cv2
from skimage.feature import local_binary_pattern

In [2]:
import os
directory_list = list()
dir_path = "C:/Users/hp/Desktop/Thesis project/dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_2
catagory_3
catagory_4
catagory_5
catagory_6
catagory_7


In [3]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    print("NN")
    print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+str(i)+'\n')
labelfile.close()

NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_0/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_1/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_2/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_3/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_4/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_5/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_6/*jpg
NN
C:/Users/hp/Desktop/Thesis project/dataset/catagory_7/*jpg


In [4]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [6]:
import cv2
import numpy as np
items = []
true_items = []
def add_dict(dic,val1,val2,val3):
    val=0
    for i in range(1,4):
        if(i==1):
            val=val1
        elif(i==2):
            val=val2
        elif(i==3):
            val=val3
        if (1000*i+val) not in dic.keys():
            dic[25000*i] = dic[25000*i] +1
        else:
            dic[i*1000+val] = dic[i*1000+val] + 1
    return 1
def add_patrn(dic,val1,val2,val3,val4):
    val=0
    for i in range(4,8):
        if(i==4):
            val=val1
        elif(i==5):
            val=val2
        elif(i==6):
            val=val3
        elif(i==7):
            val=val4
        if(1000*i+val) not in dic.keys():
            dic[25000*i]=dic[25000*i]+1
        else:
            dic[i*1000+val]=dic[i*1000+val]+1
    return 1
def lap(x,y,img):
    t=7
    sums=0
    var=0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for i in range(0,8):
        sums+=int(img[x+fx[i]][y+fy[i]])
    mean=sums/8.0
    for i in range(0,8):
        var+=int((mean-int(img[x+fx[i]][y+fy[i]]))**2)
    var=var/8.0
    var=var**(0.5)                                               
    for i in range(0, 8):
        if(int(img[x][y])+t>=int(img[x+fx[i]][y+fy[i]])):
            if(int(img[x][y])-t<=int(img[x+fx[i]][y+fy[i]])):
                out.append(1)
            else:
                out.append(3)
        else:
             out.append(2)
    return out,var,mean
def pattern(var,mean,x,y,img):
    if(var>31):
        p=60
        q=88
        fx=[0,-1,-1,-1,0,1,1,1,0]
        fy=[1,1,0,-1,-1,-1,0,1,1]
        for i in range(0, 8):
            if(int(img[x+fx[i]][y+fy[i]])+q>=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(int(img[x+fx[i]][y+fy[i]])+p<=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(1)
                else:
                    out.append(3)
            elif(int(img[x+fx[i]][y+fy[i]])-q<=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(int(img[x+fx[i]][y+fy[i]])-p>=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(2)
                else:
                    out.append(4)
    return out
def local_ap_pattern(img,dic,q):
    try:
        for x in range(2, len(img)-2):
            for y in range(2, len(img[0])-2):
                value,ab,cd = lap(x,y,img)
                patrn= pattern(ab,cd,x,y,img)
                print("Comes_under_pattern")
                val1=0
                val2=0
                val3=0
                for i in range(0,len(value)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                add_dict(dic,val1,val2,val3)
                val1=0
                val2=0
                val3=0
                val4=0
                for i in range(0,len(patrn)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                    elif(value[i]==4):
                        val4+=2**(7-i)
                add_patrn(dic,val1,val2,val3,val4)
    except TypeError as e:
        print ("type error "+str(j))
        return dic,0
    return dic,1
             

In [7]:
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
#items.append(0)
for i in range(0, 256):
    if uniform(i):
        items.append(1*1000+i)
        true_items.append(i)
        items.append(2*1000+i)
        items.append(3*1000+i)
        items.append(4*1000+i)
        items.append(5*1000+i)
        items.append(6*1000+i)
        items.append(7*1000+i)
items.append(25000)
items.append(50000)
items.append(25000*3)
items.append(25000*4)
items.append(25000*5)
items.append(25000*6)
items.append(25000*7)
true_items.append(25000)

In [8]:
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
#from keras.utils import np_utils
from scipy.stats import itemfreq
import time
from functools import wraps

df = pd.read_csv(shuffled_labels,header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0
a=-1
# uncomment below this for local tetra pattern
dic = {}
def create_dic():
    #for i in range(len(items)):
    for i in items:
        dic[i] = 0
    return 1
start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    #print(row[1][0])
    img = cv2.imread(row[1][0])
    try:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dic,a=local_ap_pattern(img,dic,j)
        new_x = []
        if(a==1):
            for i in dic.keys():
                new_x.append(dic[i])
            new_x = np.array(new_x)
            hist = new_x/np.sum(new_x)
            #print(hist)
            X_addrs.append(row[1][0])
            X_hist.append(hist)
            Y_hist.append(row[1][1])
            if(j%500==0):
                elapsed_time = time.time() - start_time
                start_time = time.time()
                print(str(j)+" done in " + str(elapsed_time))
    except Exception as e:
        print ("assertion error "+str(j))
    j = j + 1
    

assertion error 0
assertion error 1
assertion error 2
assertion error 3
assertion error 4
assertion error 5
assertion error 6
assertion error 7
assertion error 8
assertion error 9
assertion error 10
assertion error 11
assertion error 12
assertion error 13
assertion error 14
assertion error 15
assertion error 16
assertion error 17
assertion error 18
assertion error 19
assertion error 20
assertion error 21
assertion error 22
assertion error 23
assertion error 24
assertion error 25
assertion error 26
assertion error 27
assertion error 28
assertion error 29
assertion error 30
assertion error 31
assertion error 32
assertion error 33
assertion error 34
assertion error 35
assertion error 36
assertion error 37
assertion error 38
assertion error 39
assertion error 40
assertion error 41
assertion error 42
assertion error 43
assertion error 44
assertion error 45
assertion error 46
assertion error 47
assertion error 48
assertion error 49
assertion error 50
assertion error 51
assertion error 52
ass

assertion error 428
assertion error 429
assertion error 430
assertion error 431
assertion error 432
assertion error 433
assertion error 434
assertion error 435
assertion error 436
assertion error 437
assertion error 438
assertion error 439
assertion error 440
assertion error 441
assertion error 442
assertion error 443
assertion error 444
assertion error 445
assertion error 446
assertion error 447
assertion error 448
assertion error 449
assertion error 450
assertion error 451
assertion error 452
assertion error 453
assertion error 454
assertion error 455
assertion error 456
assertion error 457
assertion error 458
assertion error 459
assertion error 460
assertion error 461
assertion error 462
assertion error 463
assertion error 464
assertion error 465
assertion error 466
assertion error 467
assertion error 468
assertion error 469
assertion error 470
assertion error 471
assertion error 472
assertion error 473
assertion error 474
assertion error 475
assertion error 476
assertion error 477


KeyboardInterrupt: 

In [21]:
print(X_addrs[0])
print(X_hist[0])
print(Y_hist[0])
#print(X_hist)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(len(X_addrs))
print(len(X_hist[50]))
print(X_hist.shape)
#print((Y_hist))

import pandas as pd 
df = pd.DataFrame(X_hist)
df.to_csv("tet_pattern_normalized_data_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("tet_pattern_normalized_label_Y_hist.csv",header=None)

IndexError: list index out of range

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('tet_pattern_normalized_data_X_hist.csv', sep=',',header=None)
x = df
Y_hist= pd.read_csv('tet_pattern_normalized_label_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)
Y_hist = Y_hist[:,1]
X_hist = []
x = np.array(x)
x = x[:,1:472]
print(len(x))
print(Y_hist.shape)

In [ ]:
for i in range (x.shape[0]):
    x1 = np.array(x[i][0:177])
    x1 = x1/np.sum(x1)
    x2 = np.array(x[i][177:413])
    x2 = x2/np.sum(x2)
    new_x = x1.tolist()+x2.tolist()
    #new_x = np.array(new_x)
    X_hist.append(new_x)

X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))